In [1]:
%matplotlib inline
import torch
from IPython import display
from matplotlib import pyplot as plt
import numpy as np
import random

In [11]:
num_inputs = 2
num_examples = 1000
true_w = [2,-3.4]
true_b = 4.2
features = torch.from_numpy(np.random.normal(0,1,(num_examples,num_inputs)))
labels = true_w[0]*features[:,0]+true_w[1]*features[:,1]+true_b
labels += torch.from_numpy(np.random.normal(0,0.01,size=labels.size()))

In [12]:
def data_iter(batch_size,features,labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    random.shuffle(indices)#样本的读取顺序是随机的
    for i in range(0,num_examples,batch_size):
        j = torch.LongTensor(indices[i:min(i + batch_size,num_examples)])#最后一次可能不足一个batch
        yield features.index_select(0,j),labels.index_select(0,j)

In [14]:
batch_size = 10
for X,y in data_iter(batch_size,features,labels):
    print(X,y)
    break

tensor([[ 0.9650,  0.6990],
        [-0.9565,  0.9432],
        [ 0.2746, -0.5844],
        [ 0.0692, -0.8835],
        [ 0.9751, -0.8390],
        [ 0.5512, -0.1869],
        [ 1.8654,  0.5743],
        [ 0.3536, -0.2899],
        [ 0.5884, -0.3958],
        [-1.7572,  1.6680]], dtype=torch.float64) tensor([ 3.7455, -0.9278,  6.7362,  7.3432,  9.0025,  5.9332,  5.9883,  5.8974,
         6.7138, -4.9918], dtype=torch.float64)


In [4]:
w = torch.tensor(np.random.normal(0,0.01,(num_inputs,1)),dtype=torch.float32)
b = torch.zeros(1,dtype=torch.float32)

In [5]:
w.requires_grad_(requires_grad=True)
b.requires_grad_(requires_grad=True)

tensor([0.], requires_grad=True)

In [26]:
#使用nn函数做矩阵乘法
def linreg(X, w, b):
    return torch.mm(X, w) + b

In [21]:
def squared_loss(y_hat,y):
    return (y_hat - y.view(y_hat.size()))**2/2

In [22]:
def sgd(params,lr,batch_size):
    for param in params:
        param.data -= lr * param.grad / batch_size

In [27]:
lr = 0.03
num_epochs = 3
net = linreg
loss = squared_loss
for epoch in range(num_epochs):#训练模型一共需要num_epochs个迭代周期。
    #在每一个迭代周期中，会使用训练数据集中所有样本一次（假设样本数能够被批量大小整除）
    #和y分别是小批量样本的特征和标签
    for X, y in data_iter(batch_size,features,labels):
        l = loss(net(X,w,b), y).sum()#l是有关小批量x和y的损失
        l.backward()
        sgd([w,b],lr,batch_size)#使用小批量随机梯度下降迭代模型参数
        w.grad.data.zero_()
        b.grad.data.zero_()
    train_l = loss(net(features,w,b),labels)
    print('epoch %d,loss %f' %(epoch+1,train_l.mean().item()))

RuntimeError: Expected object of scalar type Double but got scalar type Float for argument #3 'mat2' in call to _th_addmm_out